# adding todo tools in our ReAct agent 

building on the ReAct agent from notebook 01, we'll create a simple todo management system that can:
- add new todos
- list current todos  
- mark todos as completed
- remember todos across agent interactions

this demonstrates how agents can maintain state and memory between operations.


In [1]:
# Import the ReAct agent and other dependencies from notebook 01
import json
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import Dict, List, Any

load_dotenv()

# Setup OpenAI client
openrouter_key = os.getenv("OPENROUTER_API_KEY")
openai_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_key,
)
model_name = os.getenv("OPENAI_MODEL")

# Simple in-memory todo storage
todos = []


In [2]:
def add_todo(content: str, priority: str = "medium"):
    todo_id = len(todos) + 1
    new_todo = {
        "id": todo_id,
        "content": content,
        "status": "pending",
        "priority": priority
    }
    todos.append(new_todo)
    return {"success": True, "todo": new_todo, "message": f"Added todo: {content}"}

def list_todos():
    if not todos:
        return {"todos": [], "message": "No todos found"}
    return {"todos": todos, "message": f"Found {len(todos)} todos"}

def complete_todo(todo_id: int):
    for todo in todos:
        if todo["id"] == todo_id:
            todo["status"] = "completed"
            return {"success": True, "todo": todo, "message": f"Completed todo: {todo['content']}"}
    return {"success": False, "message": f"Todo with id {todo_id} not found"}

def delete_todo(todo_id: int):
    global todos
    for i, todo in enumerate(todos):
        if todo["id"] == todo_id:
            deleted_todo = todos.pop(i)
            return {"success": True, "todo": deleted_todo, "message": f"Deleted todo: {deleted_todo['content']}"}
    return {"success": False, "message": f"Todo with id {todo_id} not found"}


In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_todo",
            "description": "Add a new todo item to the list",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {"type": "string", "description": "The todo content/description"},
                    "priority": {"type": "string", "enum": ["low", "medium", "high"], "default": "medium"}
                },
                "required": ["content"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "list_todos",
            "description": "List all current todos",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "complete_todo",
            "description": "Mark a todo as completed",
            "parameters": {
                "type": "object",
                "properties": {
                    "todo_id": {"type": "integer", "description": "The ID of the todo to complete"}
                },
                "required": ["todo_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "delete_todo", 
            "description": "Delete a todo from the list",
            "parameters": {
                "type": "object",
                "properties": {
                    "todo_id": {"type": "integer", "description": "The ID of the todo to delete"}
                },
                "required": ["todo_id"]
            }
        }
    }
]

TOOL_MAPPING = {
    "add_todo": add_todo,
    "list_todos": list_todos,
    "complete_todo": complete_todo,
    "delete_todo": delete_todo
}


In [4]:
# copy the ReAct Agent class from notebook 01 or create a new file and import it but i want it here.
class ReActAgent:
    def __init__(self, client, tools, tool_mapping, model_name):
        self.client = client
        self.tools = tools
        self.tool_mapping = tool_mapping
        self.model_name = model_name
        self.messages = []
        self.max_iterations = 50
        
    def add_system_message(self, content: str):
        self.messages.append({
            "role": "system",
            "content": content
        })
    
    def add_user_message(self, content: str):
        self.messages.append({
            "role": "user", 
            "content": content
        })
    
    def _execute_tool_call(self, tool_call):
        tool_name = tool_call.function.name
        
        try:
            tool_args = json.loads(tool_call.function.arguments)
            print(f"executing {tool_name} with args: {tool_args}")
            
            if tool_name in self.tool_mapping:
                result = self.tool_mapping[tool_name](**tool_args)
                print(f"✅ tool result: {result}")
                return result
            else:
                error_msg = f"unknown tool: {tool_name}"
                print(f"❌ {error_msg}")
                return {"error": error_msg}
                
        except Exception as e:
            error_msg = f"tool execution failed: {str(e)}"
            print(f"❌ {error_msg}")
            return {"error": error_msg}
    
    def _handle_tool_calls(self, response_message):
        if not hasattr(response_message, 'tool_calls') or not response_message.tool_calls:
            return False
            
        self.messages.append({
            "role": "assistant",
            "content": response_message.content,
            "tool_calls": response_message.tool_calls
        })
        
        for tool_call in response_message.tool_calls:
            result = self._execute_tool_call(tool_call)
            
            self.messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result, indent=2)
            })
        
        return True 
    
    def run(self, user_input: str) -> str:
        self.add_user_message(user_input)
        
        print(f"User: {user_input}")
        print("=" * 50)
        
        iteration = 0
        while iteration < self.max_iterations:
            iteration += 1
            print(f"\n🔄 Iteration {iteration}")
            
            request = {
                "model": self.model_name,
                "messages": self.messages,
                "tools": self.tools
            }
            
            completion = self.client.chat.completions.create(**request)
            response_message = completion.choices[0].message
            
            print(f"🤖 Assistant: {response_message.content}")
            
            has_tool_calls = self._handle_tool_calls(response_message)
            
            if not has_tool_calls:
                print("done with work")
                return response_message.content
        
        return "Maximum iterations reached. Task may be incomplete."


In [5]:
# create the todo agent
agent = ReActAgent(
    client=openai_client,
    tools=tools,
    tool_mapping=TOOL_MAPPING,
    model_name=model_name
)

system_prompt = """You are a helpful todo management assistant. You can help users:
- Add new todos to their list
- View all current todos
- Mark todos as completed
- Delete todos they no longer need

Be friendly and efficient. Always show the user what todos they have after making changes."""

agent.add_system_message(system_prompt)


In [6]:
result = agent.run("Add these todos: 'Buy groceries', 'Finish project report', and 'Call mom'")
print(f"final result: {result}")


User: Add these todos: 'Buy groceries', 'Finish project report', and 'Call mom'

🔄 Iteration 1
🤖 Assistant: 
executing add_todo with args: {'content': 'Buy groceries'}
✅ tool result: {'success': True, 'todo': {'id': 1, 'content': 'Buy groceries', 'status': 'pending', 'priority': 'medium'}, 'message': 'Added todo: Buy groceries'}
executing add_todo with args: {'content': 'Finish project report'}
✅ tool result: {'success': True, 'todo': {'id': 2, 'content': 'Finish project report', 'status': 'pending', 'priority': 'medium'}, 'message': 'Added todo: Finish project report'}
executing add_todo with args: {'content': 'Call mom'}
✅ tool result: {'success': True, 'todo': {'id': 3, 'content': 'Call mom', 'status': 'pending', 'priority': 'medium'}, 'message': 'Added todo: Call mom'}
executing list_todos with args: {}
✅ tool result: {'todos': [{'id': 1, 'content': 'Buy groceries', 'status': 'pending', 'priority': 'medium'}, {'id': 2, 'content': 'Finish project report', 'status': 'pending', 'prior

In [7]:
# test 2: list current todos
# create a new agent instance to demonstrate memory persistence

agent2 = ReActAgent(
    client=openai_client,
    tools=tools,
    tool_mapping=TOOL_MAPPING,
    model_name=model_name
)
agent2.add_system_message(system_prompt)

result = agent2.run("Show me all my current todos")
print(f"final result: {result}")


User: Show me all my current todos

🔄 Iteration 1
🤖 Assistant: 
executing list_todos with args: {}
✅ tool result: {'todos': [{'id': 1, 'content': 'Buy groceries', 'status': 'pending', 'priority': 'medium'}, {'id': 2, 'content': 'Finish project report', 'status': 'pending', 'priority': 'medium'}, {'id': 3, 'content': 'Call mom', 'status': 'pending', 'priority': 'medium'}], 'message': 'Found 3 todos'}

🔄 Iteration 2
🤖 Assistant: Here are your current todos:

1. **Buy groceries** (ID: 1, Priority: Medium, Status: Pending)
2. **Finish project report** (ID: 2, Priority: Medium, Status: Pending)
3. **Call mom** (ID: 3, Priority: Medium, Status: Pending)

You have 3 todos in total. Let me know if you'd like to add, complete, or delete any!
done with work
final result: Here are your current todos:

1. **Buy groceries** (ID: 1, Priority: Medium, Status: Pending)
2. **Finish project report** (ID: 2, Priority: Medium, Status: Pending)
3. **Call mom** (ID: 3, Priority: Medium, Status: Pending)

Yo

In [8]:
# test 3: complete a todo and add a new one
agent3 = ReActAgent(
    client=openai_client,
    tools=tools,
    tool_mapping=TOOL_MAPPING,
    model_name=model_name
)
agent3.add_system_message(system_prompt)

result =  agent3.run("mark todo #1 as completed and add a new todo 'Book dentist appointment'")
print(f"final result: {result}")


User: mark todo #1 as completed and add a new todo 'Book dentist appointment'

🔄 Iteration 1
🤖 Assistant: 
executing complete_todo with args: {'todo_id': 1}
✅ tool result: {'success': True, 'todo': {'id': 1, 'content': 'Buy groceries', 'status': 'completed', 'priority': 'medium'}, 'message': 'Completed todo: Buy groceries'}
executing add_todo with args: {'content': 'Book dentist appointment'}
✅ tool result: {'success': True, 'todo': {'id': 4, 'content': 'Book dentist appointment', 'status': 'pending', 'priority': 'medium'}, 'message': 'Added todo: Book dentist appointment'}

🔄 Iteration 2
🤖 Assistant: 
executing list_todos with args: {}
✅ tool result: {'todos': [{'id': 1, 'content': 'Buy groceries', 'status': 'completed', 'priority': 'medium'}, {'id': 2, 'content': 'Finish project report', 'status': 'pending', 'priority': 'medium'}, {'id': 3, 'content': 'Call mom', 'status': 'pending', 'priority': 'medium'}, {'id': 4, 'content': 'Book dentist appointment', 'status': 'pending', 'priorit